In [ ]:
# load the data
import numpy as np
import imageio
import glob
import os

path = "./data_t_v0/2024-12-06__15-33-12/img"

# Get and print the current working directory
current_path = os.getcwd()
print(f"Current working directory: {current_path}")


# Define the path

# Get a sorted list of all PNG files in the directory
png_files = sorted(glob.glob(os.path.join(path, "*.png")))
print("png_files", png_files)

# Load the images into a list
images = [imageio.imread(file) for file in png_files]

# load 

# generate video 
imageio.mimsave("tmp.mp4" , images  , fps=10., codec = "h264")


In [ ]:
# load the data
import json
from FlexivPy.robot.model.pinocchio import FlexivModel
import numpy as np
import pathlib
import pickle

data_file = "./data_t_v0/2024-12-06__15-33-12/data/data.pkl"
with open(data_file, "rb") as f:
    Din = pickle.load(f)
model = FlexivModel()

# compute the end effector position. 
def get_endeff_pos(q):
    dq = np.zeros(7)
    D = model.getInfo(q, dq)
    T0 =  D["poses"]["link7"]
    return T0


T = get_endeff_pos(Din["q"][0])



